## 변환 후 전처리

In [1]:
import json
import pandas as pd


with open('/home/yjtech2/Desktop/yurim/LLM/DATA/띄어쓰기문장부호오류.json', 'r') as f:
    json_data = json.load(f)
    
data = []
for item in json_data['data']:
    if 'annotation' in item:
        annotation = item['annotation']
        
        if 'err_sentence' in annotation and 'cor_sentence' in annotation:
            data.append({
                'err_sentence': annotation['err_sentence'],
                'cor_sentence': annotation['cor_sentence']
            })
            
df = pd.DataFrame(data)

In [2]:
from quickspacer import Spacer

spacing = Spacer()
df['res_sentence'] = df['err_sentence'].apply(lambda x: spacing.space([x]))

2024-11-13 16:45:44.596508: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-13 16:45:44.621252: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-13 16:45:44.865389: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-11-13 16:45:45.099178: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See 

In [6]:
df

,err_sentence,cor_sentence,res_sentence
0,일손과시간이부족해 검증을 다못했다는 군색한변명은안통한다,일손과 시간이 부족해 검증을 다 못했다는 군색한 변명은 안 통한다.,[일손과 시간이 부족해 검증을 다 못했다는 군색한 변명은 안통한 다]
1,총리 후보가사퇴할때마다대통령에게 오는타격은 엄청나다,총리 후보가 사퇴할 때마다 대통령에게 오는 타격은 엄청나다.,[총리 후보가 사퇴할 때마다 대통령에게 오는 타격은 엄청나 다]
2,이정부 들어 처음으로 박 대통령에대한부정적평가가 긍정적평가를 넘어선 것이다,이 정부 들어 처음으로 박 대통령에 대한 부정적 평가가 긍정적 평가를 넘어선 것이다.,[이 정부 들어 처음으로 박 대통령에 대한 부정적 평가가 긍정적 평가를 넘어선 것이 다]
3,대통령이지지율에일희일비할필요는 없다.,대통령이 지지율에 일희일비할 필요는 없다.,[대통령이 지지율에 일희 일비할 필요는 없다.]
4,위대한 업적을 남긴지도자일수록 반대여론을 무릅쓴 경우가많다,위대한 업적을 남긴 지도자일수록 반대 여론을 무릅쓴 경우가 많다.,[위대한 업적을 남긴 지도자일수록 반대 여론을 무릅쓴 경우가 많다]
...,...,...,...
35995,서울 벗어나면 놀이터가없는마을도,서울 벗어나면 놀이터가 없는 마을도,[서울 벗어나면 놀이터가 없는 마을도]
35996,집 밖으로나가 뛰어놀고싶은 어린이들에게 놀이터는 너무 멀리있거나 즐겁게 놀 만한기구...,집 밖으로 나가 뛰어놀고 싶은 어린이들에게 놀이터는 너무 멀리 있거나 즐겁게 놀 만...,[집 밖으로 나가 뛰어놀고 싶은 어린이들에게 놀이터는 너무 멀리 있거나 즐겁게 놀 ...
35997,재미가없으니 어린이들이모이지않고 같이 놀 친구가없어서 더욱 갈일이없어진다,"재미가 없으니 어린이들이 모이지 않고, 같이 놀 친구가 없어서 더욱 갈 일이 없어진다.",[재미가 없으니 어린이들이 모이지 않고 같이 놀 친구가 없어서 더욱 갈 일이 없어진 다]
35998,어린이들이 뛰어놀수있는 공간마저도 지역에따라 격차를 보이고있는것이다,어린이들이 뛰어놀 수 있는 공간마저도 지역에 따라 격차를 보이고 있는 것이다.,[어린이들이 뛰어놀 수 있는 공간마저도 지역에 따라 격차를 보이고 있는 것이 다]


In [3]:
import pandas as pd
import re
from difflib import SequenceMatcher

def add_comparison_columns(df):
    
    def calculate_spacing_similarity(row):
        """
        띄어쓰기를 포함한 두 문장 간의 유사도를 계산하는 함수
        """
        if pd.isna(row['res_sentence']) or pd.isna(row['cor_sentence']):
            return 0
        
        # 문장의 띄어쓰기를 유지한 채로 비교
        res = row['res_sentence']
        cor = row['cor_sentence']
        
        # 완전히 같으면 1 반환
        if res == cor:
            return 1.0
        
        # 다르면 띄어쓰기를 포함한 유사도 계산
        similarity = SequenceMatcher(None, res, cor).ratio()
        
        # 띄어쓰기 패턴의 정확도를 별도로 계산
        res_spaces = [i for i, char in enumerate(res) if char == ' ']
        cor_spaces = [i for i, char in enumerate(cor) if char == ' ']
        
        # 공통된 띄어쓰기 위치 수 계산
        common_spaces = len(set(res_spaces) & set(cor_spaces))
        total_spaces = len(set(res_spaces) | set(cor_spaces))
        
        # 띄어쓰기 정확도 (공통 띄어쓰기 / 전체 띄어쓰기)
        spacing_accuracy = common_spaces / total_spaces if total_spaces > 0 else 1.0
        
        # 전체 유사도와 띄어쓰기 정확도를 조합 (가중치는 조정 가능)
        final_ratio = (similarity + spacing_accuracy) / 2
        
        return round(final_ratio, 3)
    
    # check 컬럼 추가 (완전히 같으면 1, 다르면 0)
    df['check'] = (df['res_sentence'] == df['cor_sentence']).astype(int)
    
    # ratio 컬럼 추가 (띄어쓰기를 고려한 유사도 계산)
    df['ratio'] = df.apply(calculate_spacing_similarity, axis=1)
    
    return df

# 사용 예시
if __name__ == "__main__":
    processed_df = df[['err_sentence', 'res_sentence', 'cor_sentence']]
    
    def text_clean(text):
        
        if isinstance(text, list):
            text = str(text[0]) if text else ""
        elif not isinstance(text, str):
            text = str(text)
            
        # pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
        # text = re.sub(pattern, '', text)
        
        pattern = '[^\w\s]'         # 특수기호제거
        text = re.sub(pattern, '', text)

        return text 
    
    processed_df['res_sentence'] = processed_df['res_sentence'].apply(text_clean)
    processed_df['cor_sentence'] = processed_df['cor_sentence'].apply(text_clean)
    
    result_df = add_comparison_columns(processed_df)
    
    # 결과 출력
    print("비교 완료")
    print('정확도: ', (len(result_df[result_df['check'] == 1]) / len(result_df)))
    print('비율 평균: ', sum(result_df['ratio']) / len(result_df))

비교 완료
정확도:  0.62575
비율 평균:  0.9212291666666507


In [5]:
result_df

,err_sentence,res_sentence,cor_sentence,check,ratio
0,일손과시간이부족해 검증을 다못했다는 군색한변명은안통한다,일손과 시간이 부족해 검증을 다 못했다는 군색한 변명은 안통한 다,일손과 시간이 부족해 검증을 다 못했다는 군색한 변명은 안 통한다,0,0.886
1,총리 후보가사퇴할때마다대통령에게 오는타격은 엄청나다,총리 후보가 사퇴할 때마다 대통령에게 오는 타격은 엄청나 다,총리 후보가 사퇴할 때마다 대통령에게 오는 타격은 엄청나다,0,0.930
2,이정부 들어 처음으로 박 대통령에대한부정적평가가 긍정적평가를 넘어선 것이다,이 정부 들어 처음으로 박 대통령에 대한 부정적 평가가 긍정적 평가를 넘어선 것이 다,이 정부 들어 처음으로 박 대통령에 대한 부정적 평가가 긍정적 평가를 넘어선 것이다,0,0.956
3,대통령이지지율에일희일비할필요는 없다.,대통령이 지지율에 일희 일비할 필요는 없다,대통령이 지지율에 일희일비할 필요는 없다,0,0.632
4,위대한 업적을 남긴지도자일수록 반대여론을 무릅쓴 경우가많다,위대한 업적을 남긴 지도자일수록 반대 여론을 무릅쓴 경우가 많다,위대한 업적을 남긴 지도자일수록 반대 여론을 무릅쓴 경우가 많다,1,1.000
...,...,...,...,...,...
35995,서울 벗어나면 놀이터가없는마을도,서울 벗어나면 놀이터가 없는 마을도,서울 벗어나면 놀이터가 없는 마을도,1,1.000
35996,집 밖으로나가 뛰어놀고싶은 어린이들에게 놀이터는 너무 멀리있거나 즐겁게 놀 만한기구...,집 밖으로 나가 뛰어놀고 싶은 어린이들에게 놀이터는 너무 멀리 있거나 즐겁게 놀 만...,집 밖으로 나가 뛰어놀고 싶은 어린이들에게 놀이터는 너무 멀리 있거나 즐겁게 놀 만...,1,1.000
35997,재미가없으니 어린이들이모이지않고 같이 놀 친구가없어서 더욱 갈일이없어진다,재미가 없으니 어린이들이 모이지 않고 같이 놀 친구가 없어서 더욱 갈 일이 없어진 다,재미가 없으니 어린이들이 모이지 않고 같이 놀 친구가 없어서 더욱 갈 일이 없어진다,0,0.956
35998,어린이들이 뛰어놀수있는 공간마저도 지역에따라 격차를 보이고있는것이다,어린이들이 뛰어놀 수 있는 공간마저도 지역에 따라 격차를 보이고 있는 것이 다,어린이들이 뛰어놀 수 있는 공간마저도 지역에 따라 격차를 보이고 있는 것이다,0,0.949


In [9]:
result_df.to_excel('./quickspacer_result.xlsx', index = False)

## 전처리 후 변환

In [4]:
import json
import pandas as pd


with open('/home/yjtech2/Desktop/yurim/LLM/DATA/띄어쓰기문장부호오류.json', 'r') as f:
    json_data = json.load(f)
    
data = []
for item in json_data['data']:
    if 'annotation' in item:
        annotation = item['annotation']
        
        if 'err_sentence' in annotation and 'cor_sentence' in annotation:
            data.append({
                'err_sentence': annotation['err_sentence'],
                'cor_sentence': annotation['cor_sentence']
            })
            
df = pd.DataFrame(data)

In [5]:
import pandas as pd
import re
from difflib import SequenceMatcher
from quickspacer import Spacer

def add_comparison_columns(df):
    
    def calculate_spacing_similarity(row):
        """
        띄어쓰기를 포함한 두 문장 간의 유사도를 계산하는 함수
        """
        if pd.isna(row['res_sentence']) or pd.isna(row['cor_sentence']):
            return 0
        
        # 문장의 띄어쓰기를 유지한 채로 비교
        res = row['res_sentence']
        cor = row['cor_sentence']
        
        # 완전히 같으면 1 반환
        if res == cor:
            return 1.0
        
        # 다르면 띄어쓰기를 포함한 유사도 계산
        similarity = SequenceMatcher(None, res, cor).ratio()
        
        # 띄어쓰기 패턴의 정확도를 별도로 계산
        res_spaces = [i for i, char in enumerate(res) if char == ' ']
        cor_spaces = [i for i, char in enumerate(cor) if char == ' ']
        
        # 공통된 띄어쓰기 위치 수 계산
        common_spaces = len(set(res_spaces) & set(cor_spaces))
        total_spaces = len(set(res_spaces) | set(cor_spaces))
        
        # 띄어쓰기 정확도 (공통 띄어쓰기 / 전체 띄어쓰기)
        spacing_accuracy = common_spaces / total_spaces if total_spaces > 0 else 1.0
        
        # 전체 유사도와 띄어쓰기 정확도를 조합 (가중치는 조정 가능)
        final_ratio = (similarity + spacing_accuracy) / 2
        
        return round(final_ratio, 3)
    
    # check 컬럼 추가 (완전히 같으면 1, 다르면 0)
    df['check'] = (df['res_sentence'] == df['cor_sentence']).astype(int)
    
    # ratio 컬럼 추가 (띄어쓰기를 고려한 유사도 계산)
    df['ratio'] = df.apply(calculate_spacing_similarity, axis=1)
    
    return df

# 사용 예시
if __name__ == "__main__":
    def text_clean(text):
        
        # if isinstance(text, list):
        #     text = str(text[0]) if text else ""
        # elif not isinstance(text, str):
        #     text = str(text)
            
        # pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
        # text = re.sub(pattern, '', text)
        
        pattern = '[^\w\s]'         # 특수기호제거
        text = re.sub(pattern, '', text)

        return text 
    
    df['err_sentence'] = df['err_sentence'].apply(text_clean)
    df['cor_sentence'] = df['cor_sentence'].apply(text_clean)
    
    spacing = Spacer()
    df['res_sentence'] = df['err_sentence'].apply(lambda x: spacing.space([x]))
    df['res_sentence'] = df['res_sentence'].apply(text_clean)

    result_df = add_comparison_columns(df)
    
    # 결과 출력
    print("비교 완료")
    print('정확도: ', (len(result_df[result_df['check'] == 1]) / len(result_df)))
    print('비율 평균: ', sum(result_df['ratio']) / len(result_df))

비교 완료
정확도:  0.5052777777777778
비율 평균:  0.9127419166666496


In [6]:
result_df

,err_sentence,cor_sentence,res_sentence,check,ratio
0,일손과시간이부족해 검증을 다못했다는 군색한변명은안통한다,일손과 시간이 부족해 검증을 다 못했다는 군색한 변명은 안 통한다,일손과 시간이 부족해 검증을 다 못했다는 군색한 변명은 안통한 다,0,0.886
1,총리 후보가사퇴할때마다대통령에게 오는타격은 엄청나다,총리 후보가 사퇴할 때마다 대통령에게 오는 타격은 엄청나다,총리 후보가 사퇴할 때마다 대통령에게 오는 타격은 엄청나 다,0,0.930
2,이정부 들어 처음으로 박 대통령에대한부정적평가가 긍정적평가를 넘어선 것이다,이 정부 들어 처음으로 박 대통령에 대한 부정적 평가가 긍정적 평가를 넘어선 것이다,이 정부 들어 처음으로 박 대통령에 대한 부정적 평가가 긍정적 평가를 넘어선 것이 다,0,0.956
3,대통령이지지율에일희일비할필요는 없다,대통령이 지지율에 일희일비할 필요는 없다,대통령이 지지율에 일희 일비할 필요는 없다,0,0.632
4,위대한 업적을 남긴지도자일수록 반대여론을 무릅쓴 경우가많다,위대한 업적을 남긴 지도자일수록 반대 여론을 무릅쓴 경우가 많다,위대한 업적을 남긴 지도자일수록 반대 여론을 무릅쓴 경우가 많다,1,1.000
...,...,...,...,...,...
35995,서울 벗어나면 놀이터가없는마을도,서울 벗어나면 놀이터가 없는 마을도,서울 벗어나면 놀이터가 없는 마을도,1,1.000
35996,집 밖으로나가 뛰어놀고싶은 어린이들에게 놀이터는 너무 멀리있거나 즐겁게 놀 만한기구가없다,집 밖으로 나가 뛰어놀고 싶은 어린이들에게 놀이터는 너무 멀리 있거나 즐겁게 놀 만...,집 밖으로 나가 뛰어놀고 싶은 어린이들에게 놀이터는 너무 멀리 있거나 즐겁게 놀 만...,1,1.000
35997,재미가없으니 어린이들이모이지않고 같이 놀 친구가없어서 더욱 갈일이없어진다,재미가 없으니 어린이들이 모이지 않고 같이 놀 친구가 없어서 더욱 갈 일이 없어진다,재미가 없으니 어린이들이 모이지 않고 같이 놀 친구가 없어서 더욱 갈 일이 없어진 다,0,0.956
35998,어린이들이 뛰어놀수있는 공간마저도 지역에따라 격차를 보이고있는것이다,어린이들이 뛰어놀 수 있는 공간마저도 지역에 따라 격차를 보이고 있는 것이다,어린이들이 뛰어놀 수 있는 공간마저도 지역에 따라 격차를 보이고 있는 것이 다,0,0.949
